In [88]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer_GraphCodeBert = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

model_GraphCodeBert = AutoModelForMaskedLM.from_pretrained("microsoft/graphcodebert-base")

In [89]:
import ast

def getLineAssignment(tree, lineno):
    return next((node for node in ast.walk(tree) if isinstance(node, ast.Name) and node.lineno == lineno), None)

def appendIfNeeded(variables, variable):
    for var in variables:
        if var[0] == variable[0] and var[1] == variable[1]:
            return
    variables.append(variable)

def get_variables(expression):
    tree = ast.parse(expression)
    variables = []
    explored =[]
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            for node2 in ast.walk(node):
                if node2 in explored:
                    continue
                if isinstance(node2, ast.While):
                    funcName = node.name
                    for node3 in ast.walk(node2):
                        if isinstance(node3, ast.Assign):
                            nodeVariable = getLineAssignment(tree, node3.lineno).id
                            if nodeVariable == 'self':
                                continue
                            variable = [nodeVariable, funcName, node3.lineno]
                            appendIfNeeded(variables, variable)
                            explored.append(node3)
                    explored.append(node2)
                    continue
                if isinstance(node2, ast.Assign):
                    nodeVariable = getLineAssignment(tree, node2.lineno).id
                    if nodeVariable == 'self':
                        continue
                    variable = [nodeVariable, node.name, node2.lineno]
                    appendIfNeeded(variables, variable)
                    explored.append(node2)
                    continue
                if isinstance(node2, ast.arg):
                    variable = [node2.arg, node.name, node2.lineno]
                    if node2.arg == 'self':
                        continue
                    appendIfNeeded(variables, variable)
                    explored.append(node2)
                    continue
        if not isinstance(node, ast.Assign):
            continue
        if node in explored:
            continue
        nodeVariable = getLineAssignment(tree, node.lineno).id
        if nodeVariable == 'self':
            continue
        variable = [nodeVariable, None, node.lineno]
        appendIfNeeded(variables, variable)
        explored.append(node)
    return variables

def replaceVar(tree, variable, replacement):
    num = 0
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            for node2 in ast.walk(node):
                if isinstance(node2, ast.Name):
                    nodeVariable = node2.id
                    if node2.id == replacement:
                        continue
                    var = [nodeVariable, node.name, node2.lineno]
                    if var[0] == variable[0] and var[1] == variable[1]:
                        node2.id = replacement
                        num += 1
                    continue
                if isinstance(node2, ast.arg):
                    var = [node2.arg, node.name, node2.lineno]
                    if node2.arg == replacement:
                        continue
                    if var[0] == variable[0] and var[1] == variable[1]:
                        node2.arg = replacement
                        num += 1
                    continue
        if not isinstance(node, ast.Name):
            continue
        var = [node.id, None, node.lineno]
        if node.id == replacement:
            continue
        if var[0] == variable[0] and var[1] == variable[1]:
            node.id = replacement
            num += 1
    return num

In [90]:
import keyword

def isValidReplacement(varNamesInScope, replacement):
    for var in varNamesInScope:
        if var[0] == replacement:
            return False
        if keyword.iskeyword(replacement):
            return False
        if replacement[0].isdigit():
            return False
        if len(replacement) == 0:
            return False
        if not replacement.isidentifier():
            return False
    return True

In [101]:
import astunparse
import random

def renameIdentifiers(CODE):
    variables = get_variables(CODE)
    print(variables)
    tree = ast.parse(CODE)
    numLinesToTake = 4
    fill_mask_GCB = pipeline("fill-mask", model=model_GraphCodeBert, tokenizer=tokenizer_GraphCodeBert)

    for i in range(0, len(variables)):
        numMask = replaceVar(tree, variables[i], '<mask>')
        masked = astunparse.unparse(tree)
        maskedsplittolines_old = masked.splitlines()
        maskedsplittolines = []
        for line in maskedsplittolines_old:
            if line.strip() == '':
                continue
            maskedsplittolines.append(line)
        modelinput = ''
        for j in range(-numLinesToTake, numLinesToTake + 1):
            lineno = variables[i][2] - 1 + j
            if lineno >= 0 and lineno < len(maskedsplittolines):
                modelinput += maskedsplittolines[lineno] + '\n'
        candidates = fill_mask_GCB(modelinput)
        replacement = ''
        varNamesInScope = []
        for j in variables:
            if j[1] == variables[i][1] or j[1] == None:
                varNamesInScope.append(j[0])
        if numMask == 1:
            jra = [t for t in range(0, len(candidates))]
            random.shuffle(jra)
            for j in jra:
                replacement = candidates[j]['token_str'].lstrip(' ')
                if isValidReplacement(varNamesInScope, replacement):
                    break
        else:
            jra = [t for t in range(0, len(candidates))]
            random.shuffle(jra)
            for j in jra:
                kra = [t for t in range(0, len(candidates[j]))]
                random.shuffle(kra)
                for k in kra:
                    replacement = candidates[j][k]['token_str'].lstrip(' ')
                    if isValidReplacement(varNamesInScope, replacement):
                        break
        replace = ['<mask>', variables[i][1]]
        replaceVar(tree, replace, replacement)
        variables[i][0] = replacement
    print(variables)

    new_code = astunparse.unparse(tree)
    return new_code


In [102]:
file = open("experiment_data/test.py", "r")
CODE = file.read()
file.close()
print(CODE)
print("\n\n####################\n\n")
print(renameIdentifiers(CODE))

class Node:
   def __init__(self, data=None):
      self.data = data
      self.next = None
class SLinkedList:
   def __init__(self):
      self.head = None

   def Atbegining(self, data_in):
      NewNode = Node(data_in)
      NewNode.next = self.head
      self.head = NewNode

# Function to remove node
   def RemoveNode(self, Removekey):
      HeadVal = self.head
         
      if (HeadVal is not None):
         if (HeadVal.data == Removekey):
            self.head = HeadVal.next
            HeadVal = None
            return
      while (HeadVal is not None):
         if HeadVal.data == Removekey:
            break
         prev = HeadVal
         HeadVal = HeadVal.next

      if (HeadVal == None):
         return

      prev.next = HeadVal.next
      HeadVal = None

   def LListprint(self):
      printval = self.head
      while (printval):
         print(printval.data),
         printval = printval.next

llist = SLinkedList()
llist.Atbegining("Mon")
llist.Atbegining("Tue")
llist.A